<img src='../img/EU-Copernicus-EUM_banner_logo.png' align='right' width='50%'></img>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_OLCI_advanced_data_access_hda.ipynb">OLCI advanced data access; WEkEO HDA >></a>

Copernicus Marine Training Service <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1>Learn OLCI: advanced</h1></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **None**

It is recommended to go through these modules before you start with this module.
- **<a href="../1_OLCI_introductory/1_1_OLCI_data_access.ipynb">1_1_OLCI_data_access</a>**

</div>


<hr>

<div class="alert alert-info" role="alert">

## OLCI: customised downloading from the EUMETSAT Data Store
    
</div>

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for OLCI products in the EUMETSAT Data Store using the `eumdac client`
* How to <font color="#138D75">**download**</font> components of products
* How to pre-screen downloads based on flags
* How to download products based on a match-up file

### Outline

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Example 1: Filter by collections](#filter_by_collection)
1. [Example 2: Filter by time](#filter_by_time)
1. [Example 3: Filter by space and time](#filter_by_space_and_time)
1. [Example 4: Download by component](#download_component)
1. [Example 5: Filter by flag](#filter_by_flag)

<hr>

In [ ]:
import os
import json
import datetime
import shutil
import eumdac
import validation_tools as vt
import inspect
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import eumartools
from shapely import geometry, vectorized

# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

# set collection ID for OLCI L2 WFR
collectionID = 'EO:EUM:DAT:0407'

<div class="alert alert-info" role="alert">

## <a id='filter_by_collection'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

In [ ]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

<div class="alert alert-info" role="alert">

## <a id='filter_by_time'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

In [ ]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
print(latest)

In [ ]:
# time filter the collection for products
start = datetime.datetime(2022, 1, 23, 16, 0)
end = datetime.datetime(2022, 1, 23, 16, 15)
products = selected_collection.search(dtstart=start, dtend=end)

for product in products:
    print(product)

<div class="alert alert-info" role="alert">

## <a id='filter_by_space_and_time'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

In [ ]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
start = datetime.datetime(2022, 1, 23)
end = datetime.datetime(2022, 1, 24)
roi = [[-1.0, -1.0], [4.0, -4.0], [8.0, -2.0], [9.0, 2.0], [6.0, 4.0], [1.0, 5.0], [-1.0, -1.0]]

products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
    dtstart=start, 
    dtend=end)

for product in products:
    print(product)

<div class="alert alert-info" role="alert">

## <a id='download_component'></a>Example 4: Downloading by component
[Back to top](#TOC_TOP)

</div>

In [ ]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()

for entry in latest.entries:
    if 'xfdumanifest.xml' in entry:
        with latest.open(entry=entry) as fsrc, open(fsrc.name, mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of file {fsrc.name} finished.')

<div class="alert alert-info" role="alert">

## <a id='filter_by_flag'></a>Example 5: Filtering by flag
[Back to top](#TOC_TOP)

</div>

As we can choose to download a single component, we can download only the flags and use the information contained in it to decide if we want the whole product.

In [ ]:
# Set the geometry for our ROI
roi = [[6.0, -10.0], [8.0, -10.0], [8.0, -8.0], [6.0, -8.0], [6.0, -10.0]]

In [ ]:
selected_collection = datastore.get_collection(collectionID)
# Get the latest product in a collection that matches this ROI
latest = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),).first()

# Get the flag product (for OLCI L2 this is wqsf.nc, for OLCI L1 this is qualityFlags.nc) and coordinates
for entry in latest.entries:
    if entry == 'wqsf.nc' or entry == 'geo_coordinates.nc':
        with latest.open(entry=entry) as fsrc, open(fsrc.name, mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of file {fsrc.name} finished.')

In [ ]:
# Read in the coordinate data and build a spatial mask
geo_fid = xr.open_dataset('geo_coordinates.nc')
lat = geo_fid.get('latitude').data
lon = geo_fid.get('longitude').data
geo_fid.close()

point_mask = vectorized.contains(geometry.Polygon(roi), lon,lat)

In [ ]:
# Now check the flag content for our polygon
flag_file = 'wqsf.nc'
flag_variable = 'WQSF'
flags_to_use = ['CLOUD']
flag_mask = eumartools.flag_mask(flag_file, flag_variable, flags_to_use)

In [ ]:
# Now find the union of the spatil and flag mask
fig1, (ax1, ax2) = plt.subplots(1, 2,figsize=(10, 10), dpi=150)
ax1.imshow(flag_mask, interpolation="none")
ax2.imshow(point_mask*flag_mask, interpolation="none")
plt.show()

In [ ]:
# Now check the % flag cover in our ROI. Then you can decide if you want the full product
pc_cover = int(np.sum(point_mask*flag_mask)/np.sum(point_mask)*100)
print(f'Percent flag cover {pc_cover}%')

In [ ]:
pc_cover_threshold = 40
if pc_cover > pc_cover_threshold:
    with latest.open() as fsrc, open(fsrc.name, mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
else:
    print(f'Percent flag cover > threshold ({pc_cover_threshold}%), skipping download')

<hr>

<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>